# Dev 04 - Coordinate Array Transformation

Transforming array of spherical coordinate (r, a, e) to arrays of Cartesian coordinate (x, y, z) to earth coordinate (lon, lat)

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('./darkmode.style')

In [ ]:
# Radar location, approximate from Google Maps: 39.7294019,140.0440751
olat, olon = 39.7294019, 140.0440751

In [ ]:
r, a = np.meshgrid(np.arange(0, 300), np.deg2rad(np.arange(0, 361)))

print('r =\n\n{}'.format(r))
print('\n')
print('a =\n\n{}'.format(a))

In [ ]:
e = 1.5 / 180 * np.pi
x = r * np.cos(e) * np.sin(a)
y = r * np.cos(e) * np.cos(a)
z = r * np.sin(e)

In [ ]:
plt.close()

In [ ]:
def cart2geo(x, y, z, olat, olon):
    e = 6371e3
    z += e
    # Step 1 - rotate the earth about the x-axis by (90 - lat)
    # Step 2 - rotate the earth about the z-axis by (90 + lon)
    # rx = np.array([[1, 0, 0], [0, np.cos(ax), -np.sin(ax)], [0, np.sin(ax), np.cos(ax)]])
    # rz = np.array([[np.cos(az), -np.sin(az), 0], [np.sin(az), np.cos(az), 0], [0, 0, 1]])
    # q = np.array([x, y, z + e])
    # p = np.matmul(rz, np.matmul(rx, q))
    # x, y, z = p
    ax = np.deg2rad(90 - olat)
    xx = x
    xy = np.cos(ax) * y - np.sin(ax) * z
    xz = np.sin(ax) * y + np.cos(ax) * z
    az = np.deg2rad(90 + olon)
    x = np.cos(az) * xx - np.sin(az) * xy
    y = np.sin(az) * xx + np.cos(az) * xy
    z = xz
    h = np.sqrt(x * x + y * y)
    lam = np.arctan2(y, x)
    phi = np.arctan2(z, h)
    lat, lon = np.rad2deg(phi), np.rad2deg(lam)
    return lat, lon

In [ ]:
lat, lon = cart2geo(x, y, z, olat, olon)

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(7, 7), dpi=120)

axs[0, 0].pcolormesh(x, y, r)
axs[0, 0].set_aspect('equal', 'box')
axs[0, 0].set_title('r')

axs[0, 1].pcolormesh(x, y, a)
axs[0, 1].set_aspect('equal', 'box')
axs[0, 1].set_title('a')

axs[0, 2].pcolormesh(x, y, e * np.ones(x.shape))
axs[0, 2].set_aspect('equal', 'box')
axs[0, 2].set_title('e')

axs[1, 0].pcolormesh(x, y, x)
axs[1, 0].set_aspect('equal', 'box')
axs[1, 0].set_title('x')

axs[1, 1].pcolormesh(x, y, y)
axs[1, 1].set_aspect('equal', 'box')
axs[1, 1].set_title('y')

axs[1, 2].pcolormesh(x, y, z)
axs[1, 2].set_aspect('equal', 'box')
axs[1, 2].set_title('z')

axs[2, 0].pcolormesh(x, y, lon)
axs[2, 0].set_aspect('equal', 'box')
axs[2, 0].set_title('lon')

axs[2, 1].pcolormesh(x, y, lat)
axs[2, 1].set_aspect('equal', 'box')
axs[2, 1].set_title('lat')

axs[2, 2].pcolormesh(x, y, np.ones(x.shape))
axs[2, 2].set_aspect('equal', 'box')
axs[2, 2].set_title('-')

fig.tight_layout()